In [1]:
import os
from pathlib import Path
from collections import Counter

import pandas as pd
from wordcloud import WordCloud
from konlpy.tag import Twitter
from dotenv import load_dotenv
from sqlalchemy import text, create_engine

In [2]:
dotenv_path = Path('..') / '.env'
load_dotenv(dotenv_path=dotenv_path)

db_host = os.environ["DB_HOST"]
db_user = os.environ['DB_USER']
db_password = os.environ['DB_PASSWORD']
db_name = os.environ['DB_NAME']

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:5432/{db_name}')

In [5]:
query = 'SELECT * FROM review_raw'

# 데이터 다운로드
df = pd.read_sql(query, engine)

# 데이터 확인
display(df)
df.info()

,user_name,keywords,review_text,review_dt,raw_text,chef_name,store_name,scraped_at
0,hyemini1112,"{""예약 후 이용"",""대기 시간 2시간 이상"",데이트,연인・배우자}",대부분 정지선쉐프님 보고 기대를 품고 방문하는 맛집이 아닐까 싶어요~방문한 날 정지...,11.6.수,hyemini1112리뷰 69사진 234팔로우이전다음예약 후 이용대기 시간 2시간 ...,정지선,티엔미미,2024-11-10 09:31:59.748722
1,lemontic1227,"{""예약 후 이용"",""대기 시간 1시간 이상"",친목,지인・동료}",화요일 오전 9시 50분쯤 도착! 직원분이 나오셔서 캐치 테이블 앱으로 예약을 받아...,11.5.화,lemontic1227리뷰 484사진 654팔로워 24팔로우동영상동영상동영상이전다음...,정지선,티엔미미,2024-11-10 09:31:59.748722
2,냠냠쩝쩝박사님,"{""대기 시간 2시간 이상""}","진챠 딤섬, 마늘새우찜 미친넘 .. 정지선셰프님 보고 갔는데 후회란 없다 또 가고싶...",10.18.금,"냠냠쩝쩝박사님리뷰 1,028사진 1,078팔로워 85팔로우이전다음대기 시간 2시간 ...",정지선,티엔미미,2024-11-10 09:31:59.748722
3,킹보비,"{""예약 없이 이용"",""대기 시간 2시간 이상"",데이트,연인・배우자}",✔️소룡포 육즙이 팡팡. 생강 곁들여 먹으니까 향이 입안에 은은하게 퍼져서 맛있음....,11.2.토,킹보비리뷰 429사진 522팔로워 20팔로우이전다음예약 없이 이용대기 시간 2시간 ...,정지선,티엔미미,2024-11-10 09:31:59.748722
4,짜유의영혼,"{""예약 후 이용"",""대기 시간 30분 이상"",데이트,연인・배우자}",미미세트 먹었어용! 개인적으로 입맛에 딱 맞는 건 아니였지만 음식들 자체가 퀄리티 ...,10.20.일,짜유의영혼리뷰 63사진 46팔로워 4팔로우이전다음예약 후 이용대기 시간 30분 이상...,정지선,티엔미미,2024-11-10 09:31:59.748722
...,...,...,...,...,...,...,...,...
1993,푸디필,"{""예약 후 이용"",""대기 시간 30분 이내"",""데이트, 친목"",""연인・배우자, 친구""}",워낙 유명한 쉐프님.. 사브에서도 너무 맛있게 먹었던 기억이 있습니다. 새로 포노 ...,5.14.화,푸디필리뷰 97사진 553팔로워 47팔로우\n\n\n\n동영상\n\n\n\n동영상이...,히든천재,포노부오노,2024-11-10 09:41:54.381752
1994,바텐더잔다르크,"{""예약 후 이용"",""대기 시간 바로 입장"",일상,친구}",최근에 갔던 이탈리안중에 최고 화덕브래드랑 치즈 프로슈토 삼합으로 꼭 시켜서 드셔보...,7.13.토,바텐더잔다르크리뷰 264사진 166팔로워 1팔로우\n\n\n\n동영상예약 후 이용대...,히든천재,포노부오노,2024-11-10 09:41:54.381752
1995,BbGDA,"{""예약 후 이용"",""대기 시간 바로 입장"",데이트,연인・배우자}",넘 맛있어요 양갈비도 먹고 참외 소르베도 먹었는데 사진이 없네요 ㅠㅠ 재방문 의사 ...,6.6.목,BbGDA리뷰 104사진 28팔로워 9팔로우이전다음예약 후 이용대기 시간 바로 입장...,히든천재,포노부오노,2024-11-10 09:41:54.381752
1996,로사리아1425,"{""예약 후 이용""}",음식도 와인도 너무 좋았습니다. 지인들이 추천하는 이유가 있네요 :),7.19.금,로사리아1425리뷰 406사진 343팔로워 59팔로우예약 후 이용음식도 와인도 너무...,히든천재,포노부오노,2024-11-10 09:41:54.381752


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_name    1998 non-null   object        
 1   keywords     1998 non-null   object        
 2   review_text  1998 non-null   object        
 3   review_dt    1998 non-null   object        
 4   raw_text     1998 non-null   object        
 5   chef_name    1998 non-null   object        
 6   store_name   1998 non-null   object        
 7   scraped_at   1998 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 125.0+ KB


In [6]:
# from glob import glob

# df = pd.concat([pd.read_parquet(path) for path in glob("../outputs/temp_*.parquet")])

In [7]:
twitter = Twitter()

/opt/homebrew/lib/python3.11/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [8]:
for store_name, group in df.groupby("store_name"):
    review_nouns = []
    for review_text in group.review_text:
        target_words = [word for (word, tag) in twitter.pos(review_text) if tag == "Noun"]
        review_nouns += target_words

    # 가장 많이 나온 단어부터 40개를 저장한다.
    counts = Counter(review_nouns)
    tags = counts.most_common(40) 

    # WordCloud를 생성한다.
    # 한글을 분석하기위해 font를 한글로 지정해주어야 된다. macOS는 .otf , window는 .ttf 파일의 위치를
    # 지정해준다. (ex. '/Font/GodoM.otf')
    wc = WordCloud(font_path="../resources/NanumFontSetup_OTF_GOTHIC/NanumGothicBold.otf",background_color="white", max_font_size=60)
    cloud = wc.generate_from_frequencies(dict(tags))

    cloud.to_file(f'../outputs/{store_name}_cloud.jpg')

In [16]:
store_name = df.store_name.values[0]